<a href="https://colab.research.google.com/github/0su1327/Graduation_Thesis/blob/main/One_hot_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import csv
import pandas as pd
import cv2
import matplotlib.pyplot as plt


# MAX_COLMUM = 5
# MAX_ITEM = 20
# AREA_INDEX = 2
# AREA_INDEX2 = 4

dataset = pd.read_csv("/content/drive/MyDrive/colab_refer_data/kdd_test.csv", header=0)

col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty_level"]

dataset_train = pd.read_csv("/content/drive/MyDrive/colab_refer_data/kdd_test.csv", header=None, names = col_names)
dataset_test = pd.read_csv("/content/drive/MyDrive/colab_refer_data/kdd_train.csv", header=None, names = col_names)


In [ ]:
# feature 추출을 통해서 weight가 높은 features 13개 적용

# x1의 경우 nominal type 데이터로서(범주형 문자형 데이터)
# 정수형 인코딩 후 one hot vector로 변환해야함(근데 아직 정수형 인코딩 안함)

# xy의 경우 binary, numeric type 두 가지로 나눌 수 있는데 이 경우는
# 속성들간의 차이가 많이 나는 경우가 있어서 최소 최대 정규화를 통해서
# 데이터를 0과 1사이에 scaling을 시킨다.


x1 = dataset_train[['protocol_type','service','flag','label']]
xy = dataset_train[['src_bytes','dst_host_count','difficulty_level','dst_bytes','dst_host_srv_diff_host_rate','wrong_fragment','dst_host_same_src_port_rate','hot',
              'logged_in','dst_host_rerror_rate']]
print(x1)

      protocol_type   service  flag    label
0               tcp   private   REJ  neptune
1               tcp   private   REJ  neptune
2               tcp  ftp_data    SF   normal
3              icmp     eco_i    SF    saint
4               tcp    telnet  RSTO    mscan
...             ...       ...   ...      ...
22539           tcp      smtp    SF   normal
22540           tcp      http    SF   normal
22541           tcp      http    SF     back
22542           udp  domain_u    SF   normal
22543           tcp    sunrpc   REJ    mscan

[22544 rows x 4 columns]


In [ ]:
x1_dum = pd.get_dummies(x1)
x1_dum.head()

,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_auth,service_bgp,service_courier,service_csnet_ns,...,label_snmpgetattack,label_snmpguess,label_sqlattack,label_teardrop,label_udpstorm,label_warezmaster,label_worm,label_xlock,label_xsnoop,label_xterm
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
Fvote = pd.concat([x1_dum, xy], axis='columns')
Fvote.head()

,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_auth,service_bgp,service_courier,service_csnet_ns,...,src_bytes,dst_host_count,difficulty_level,dst_bytes,dst_host_srv_diff_host_rate,wrong_fragment,dst_host_same_src_port_rate,hot,logged_in,dst_host_rerror_rate
0,0,1,0,0,0,0,0,0,0,0,...,0,255,21,0,0.00,0,0.00,0,0,1.00
1,0,1,0,0,0,0,0,0,0,0,...,0,255,21,0,0.00,0,0.00,0,0,1.00
2,0,1,0,0,0,0,0,0,0,0,...,12983,134,21,0,0.02,0,0.61,0,0,0.00
3,1,0,0,0,0,0,0,0,0,0,...,20,3,15,0,0.28,0,1.00,0,0,0.00
4,0,1,0,0,0,0,0,0,0,0,...,0,29,11,15,0.02,0,0.03,0,0,0.83


In [ ]:
Fvote.to_csv('Fvote.csv', index=True, sep=',', encoding='utf-8')

In [ ]:
data = pd.read_csv("Fvote.csv", header=0)
data.head()

,Unnamed: 0,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_auth,service_bgp,service_courier,...,src_bytes,dst_host_count,difficulty_level,dst_bytes,dst_host_srv_diff_host_rate,wrong_fragment,dst_host_same_src_port_rate,hot,logged_in,dst_host_rerror_rate
0,0,0,1,0,0,0,0,0,0,0,...,0,255,21,0,0.00,0,0.00,0,0,1.00
1,1,0,1,0,0,0,0,0,0,0,...,0,255,21,0,0.00,0,0.00,0,0,1.00
2,2,0,1,0,0,0,0,0,0,0,...,12983,134,21,0,0.02,0,0.61,0,0,0.00
3,3,1,0,0,0,0,0,0,0,0,...,20,3,15,0,0.28,0,1.00,0,0,0.00
4,4,0,1,0,0,0,0,0,0,0,...,0,29,11,15,0.02,0,0.03,0,0,0.83


In [ ]:
# 튀는 값에 한해서는 최소 최대 정규화를 통해서 데이터를 정규화한다.
# scaling을 통한 튀는값을 줄인다. 0과 1 사이의 값으로 scaling 한다.

# min max normalization logic

# min_value = np.min(xy)
# max_value = np.max(xy)

# output = (xy - min_value) / (max_value - min_value)

# # subplot(row, col, index)
# plt.subplot(2, 1, 1)
# plt.hist(np.ravel(output), bins=32)

# plt.show()

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

normalized_data = scaler.fit_transform(xy)

print(normalized_data)


[[0.00000000e+00 1.00000000e+00 1.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [0.00000000e+00 1.00000000e+00 1.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [2.06651271e-04 5.25490196e-01 1.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [8.68116792e-04 1.00000000e+00 7.14285714e-01 ... 1.98019802e-02
  1.00000000e+00 7.00000000e-02]
 [6.68516782e-07 1.00000000e+00 1.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 6.66666667e-01 ... 0.00000000e+00
  0.00000000e+00 4.40000000e-01]]


In [ ]:
new_Fvote = pd.concat([x1_dum, normalized_data], axis='columns')
new_Fvote.head()

TypeError: ignored